In [1]:
!pip install datasets
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, TFDistilBertModel, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from datasets import Dataset


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00


In [2]:
# Load your data
data = pd.read_csv('Classified_Dataset.csv')

# Display first few rows to verify
print(data.head())
# data_eval = pd.read_csv('Dataset-1-Evaluation.csv')
# print(data_eval.head())

                          xi  \
0    The difficulties of the   
1     Finally I have time to   
2         Diplomacy is to do   
3  Sami even referred to his   
4           I think I should   

                                                  xj             model  
0  finding a perfect balance between work and per...       llama3.2:1b  
1                      Finally i start this project.        qwen2.5:3b  
2   negotiate effectively with all parties involved.       llama3.2:1b  
3  Sami even referred to his friends as being his...  mistral-openorca  
4                                                  .  mistral-openorca  


In [3]:
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['model'])
# data['label_encoded'] = label_encoder.fit_transform(data['LLM'])

# Mapping of labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print('Label Mapping:', label_mapping)

Label Mapping: {'codegemma': 0, 'granite-code': 1, 'llama3.2:1b': 2, 'mistral-openorca': 3, 'phi3.5': 4, 'qwen2.5:3b': 5}


In [4]:
X_train_xi, X_test_xi, X_train_xj, X_test_xj, y_train, y_test = train_test_split(
    data['xi'], data['xj'], data['label_encoded'], test_size=0.2, random_state=42, stratify=data['label_encoded']
)
# X_train_xi, X_test_xi, X_train_xj, X_test_xj, y_train, y_test = train_test_split(
    # data['x_i'], data['x_j'], data['label_encoded'], test_size=0.2, random_state=42, stratify=data['label_encoded']
# )

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
MAX_SEQ_LEN = 128  # Adjust as needed based on data

def tokenize_separately(texts_xi, texts_xj):
    texts_xi = [str(text) for text in texts_xi]
    texts_xj = [str(text) for text in texts_xj]
    dataset_xi = Dataset.from_dict({"text": texts_xi})
    dataset_xj = Dataset.from_dict({"text": texts_xj})
   # dataset_xi = dataset_xi.cast_column("text",  dataset_xi.features["text"].dtype)
   # dataset_xj = dataset_xj.cast_column("text",  dataset_xj.features["text"].dtype)

    encodings_xi = tokenizer(
        texts_xi,
        add_special_tokens=True,
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    encodings_xj = tokenizer(
        texts_xj,
        add_special_tokens=True,
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    return encodings_xi, encodings_xj

In [7]:
train_encodings_xi, train_encodings_xj = tokenize_separately(X_train_xi.tolist(), X_train_xj.tolist())
test_encodings_xi, test_encodings_xj = tokenize_separately(X_test_xi.tolist(), X_test_xj.tolist())
print(train_encodings_xi)
print(train_encodings_xj)


{'input_ids': <tf.Tensor: shape=(24000, 128), dtype=int32, numpy=
array([[ 101, 3419, 2409, ...,    0,    0,    0],
       [ 101, 1045, 2371, ...,    0,    0,    0],
       [ 101, 1045, 2354, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 9511, ...,    0,    0,    0],
       [ 101, 2008, 6533, ...,    0,    0,    0],
       [ 101, 3419, 2038, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(24000, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(24000, 128), dtype=int32, numpy=
array([[  101,  3288,  2014, ...,     0,     0,     0],
       [  101,  2673,  2018, ...,     0,     0,     0],
       [  101,  1045,  2354, ...,     0,     0,     0],
       ...,
       [  101, 25430, 25164, ...,     0,     0,     0],
       [  101,

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids_xi': train_encodings_xi['input_ids'],
        'attention_mask_xi': train_encodings_xi['attention_mask'],
        'input_ids_xj': train_encodings_xj['input_ids'],
        'attention_mask_xj': train_encodings_xj['attention_mask']
    },
    y_train
)).shuffle(len(X_train_xi)).batch(32) # batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids_xi': test_encodings_xi['input_ids'],
        'attention_mask_xi': test_encodings_xi['attention_mask'],
        'input_ids_xj': test_encodings_xj['input_ids'],
        'attention_mask_xj': test_encodings_xj['attention_mask']
    },
    y_test
)).batch(32)


In [9]:
input_ids_xi = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids_xi')
attention_mask_xi = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask_xi')

input_ids_xj = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input_ids_xj')
attention_mask_xj = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='attention_mask_xj')


In [10]:
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [11]:
# Process xi
outputs_xi = bert_model(input_ids=input_ids_xi, attention_mask=attention_mask_xi)
# pooled_output_xi = outputs_xi.pooler_output
pooled_output_xi = outputs_xi.last_hidden_state[:, 0, :]


# Process xj
outputs_xj = bert_model(input_ids=input_ids_xj, attention_mask=attention_mask_xj)
# pooled_output_xj = outputs_xj.pooler_output
pooled_output_xj = outputs_xj.last_hidden_state[:, 0, :]


In [12]:
combined_output = tf.keras.layers.Concatenate()([pooled_output_xi, pooled_output_xj])


In [13]:
dense = tf.keras.layers.Dense(256, activation='relu')(combined_output)
dropout = tf.keras.layers.Dropout(0.1)(dense)
output = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(dropout)


In [14]:
model = tf.keras.Model(
    inputs=[input_ids_xi, attention_mask_xi, input_ids_xj, attention_mask_xj],
    outputs=output
)



In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [17]:
EPOCHS = 2  # Adjust based on your needs

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS
)


Epoch 1/2
750/750 [==============================] - 641s 855ms/step - loss: 0.7430 - accuracy: 0.7207 - val_loss: 0.7093 - val_accuracy: 0.7317
Epoch 2/2
750/750 [==============================] - 605s 807ms/step - loss: 0.5398 - accuracy: 0.8003 - val_loss: 0.7167 - val_accuracy: 0.7400


In [18]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


188/188 [==============================] - 51s 268ms/step - loss: 0.7167 - accuracy: 0.7400
Test Loss: 0.7166805267333984
Test Accuracy: 0.7400000095367432


In [22]:
import numpy as np
from sklearn.metrics import classification_report

# Assuming `y_test` is your true labels and `y_pred` is your predicted labels
# Replace `y_test` and `y_pred` with your actual variable names
y_pred = model.predict(test_dataset)  # Use test_dataset instead of test_data

# Extract true labels from the dataset
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred, axis=1) #  Get the class with the highest probability

# Generate and display the classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

188/188 [==============================] - 50s 267ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.64      0.68      1000
           1       0.61      0.72      0.66      1000
           2       0.73      0.81      0.77      1000
           3       0.68      0.61      0.65      1000
           4       0.90      0.89      0.89      1000
           5       0.80      0.77      0.78      1000

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000

